In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 
import edward as ed

C:\Users\Amalia\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# Model structure was taken from edward Bayesian network tutorial , http://edwardlib.org/

In [2]:
from edward.models import Normal, Bernoulli

In [3]:
def neural_network(x, W_0, W_1, W_2, b_0, b_1, b_2):
    
    h = tf.nn.tanh(tf.matmul(x, W_0) + b_0)
    h = tf.nn.tanh(tf.matmul(h, W_1) + b_1)
    h = tf.matmul(h, W_2) + b_2
    return tf.reshape(h, [-1])

In [9]:
x=pd.read_csv('Xdf').values
y=pd.read_csv('Ydf').values

In [32]:
H=2
D=9
x_train=np.zeros(5000).reshape(500,10)

In [33]:
with tf.name_scope('model'):
    W_0 = Normal(loc=tf.zeros([D, H], name="weights/loc1"), scale=tf.ones([D, H], name="weights/scale1"))
    W_1 = Normal(loc=tf.zeros([H, H], name="weights/loc2"), scale=tf.ones([H, H], name="weights/scale2"))
    W_2 = Normal(loc=tf.zeros([H, 1], name="weights/loc3"), scale=tf.ones([H, 1], name="weights/scale3"))
    b_0 = Normal(loc=tf.zeros(H, name="bias/loc1"), scale=tf.ones(H, name="bias/scale1"))
    b_1 = Normal(loc=tf.zeros(H, name="bias/loc2"), scale=tf.ones(H, name="bias/scale2"))
    b_2 = Normal(loc=tf.zeros(1, name="bias/loc3"), scale=tf.ones(1, name="bias/loc3"))

In [ ]:
x = tf.cast(X_train, dtype=tf.float32)
y = Bernoulli(logits=neural_network(x, W_0, W_1, W_2, b_0, b_1, b_2))

In [36]:
with tf.name_scope("posterior"):
    qW_0 = Normal(loc=tf.Variable(tf.random_normal([D, H], name='qw/loc1',)),
scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H], name="qw/unconstrained_scale1"))))
    qW_1 = Normal(loc=tf.Variable(tf.random_normal([H, H], name='qw/loc2')),
scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H], name="qw/unconstrained_scale2"))))
    qW_2 = Normal(loc=tf.Variable(tf.random_normal([H, 1], name='qw/loc3')),
scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, 1], name="qw/unconstrained_scale3"))))
    qb_0 = Normal(loc=tf.Variable(tf.random_normal([H], name='qw/loc4')),
scale=tf.nn.softplus(tf.Variable(tf.random_normal([H], name="qw/unconstrained_scale4"))))
    qb_1 = Normal(loc=tf.Variable(tf.random_normal([H], name='qw/loc5')),
scale=tf.nn.softplus(tf.Variable(tf.random_normal([H], name="qw/unconstrained_scale5"))))
    qb_2 = Normal(loc=tf.Variable(tf.random_normal([1], name='qw/loc6')),
scale=tf.nn.softplus(tf.Variable(tf.random_normal([1], name="qw/unconstrained_scale6"))))

In [ ]:
a=y_train.ravel()
a.shape

In [ ]:
inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
W_1: qW_1, b_1: qb_1,
W_2: qW_2, b_2: qb_2}, data={y: y_train})

In [38]:
inference.run(n_iter=1000, n_print=100, n_samples=10, logdir='log')

In [ ]:
y_post = ed.copy(y, {W_0: qW_0.mean(), b_0: qb_0.mean(),
W_1: qW_1.mean(), b_1: qb_1.mean(),
W_2: qW_2.mean(), b_1: qb_2.mean()})

In [ ]:
print(ed.evaluate('binary_accuracy', data={x:X_train, y_post:a}, output_key=y_post))


In [ ]:
print(ed.evaluate('log_lik', data={x: X_train, y_post: a}, output_key=y_post ))

In [ ]:
writer=tf.summary.FileWriter('log')
writer.add_graph(tf.Session().graph)
writer.close()